In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install PyPDF2
!pip install pdfminer
!pip install pdfplumber
!pip install pytesseract
!pip install pypdf

In [ ]:
# To read the PDF
import PyPDF2
# To analyze the PDF layout and extract text
from pdfminer.high_level import extract_pages, extract_text
from pdfminer.layout import LTTextContainer, LTChar, LTRect, LTFigure
# To extract text from tables in PDF
import pdfplumber
import pytesseract
import nltk
from nltk.tokenize import sent_tokenize
nltk.download("punkt")
import pandas as pd
# To remove the additional created files
import os
import glob
from pypdf import PdfReader


In [ ]:
%cd /content/drive/My Drive/Colab Notebooks/


In [ ]:
# Create a function to extract text

def text_extraction(element):
    # Extracting the text from the in-line text element
    line_text = element.get_text()
    # Find the formats of the text
    # Initialize the list with all the formats that appeared in the line of text
    line_formats = []
    for text_line in element:
        if isinstance(text_line, LTTextContainer):
            # Iterating through each character in the line of text
            for character in text_line:
                if isinstance(character, LTChar):
                    # Append the font name of the character
                    line_formats.append(character.fontname)
                    # Append the font size of the character
                    line_formats.append(character.size)
    # Find the unique font sizes and names in the line
    format_per_line = list(set(line_formats))

    # Return a tuple with the text in each line along with its format
    return (line_text, format_per_line)

In [ ]:

# This is formatted as code

# Find the PDF path
#pdf_path = 'Apple_Earnings.pdf'

def extract_pdf(path_pdf):
# create a PDF file object
  pdfFileObj = open(path_pdf, 'rb')
# create a PDF reader object
  pdfReaded = PyPDF2.PdfReader(pdfFileObj)
# Create the dictionary to extract text from each image
  text_per_page = {}
# We extract the pages from the PDF
  for pagenum, page in enumerate(extract_pages(path_pdf)):
    # Initialize the variables needed for the text extraction from the page
    pageObj = pdfReaded.pages[pagenum]
    page_text = []
    line_format = []
    text_from_images = []
    text_from_tables = []
    page_content = []
    # Initialize the number of the examined tables
    table_num = 0
    first_element= True
    table_extraction_flag= False
    # Open the pdf file
    pdf = pdfplumber.open(path_pdf)
    # Find the examined page
    page_tables = pdf.pages[pagenum]
    # Find the number of tables on the page
    tables = page_tables.find_tables()


    # Find all the elements
    page_elements = [(element.y1, element) for element in page._objs]
    # Sort all the elements as they appear in the page
    page_elements.sort(key=lambda a: a[0], reverse=True)

    # Find the elements that composed a page
    for i,component in enumerate(page_elements):
        # Extract the position of the top side of the element in the PDF
        pos= component[0]
        # Extract the element of the page layout
        element = component[1]

        # Check if the element is a text element
        if isinstance(element, LTTextContainer):
            # Check if the text appeared in a table
            if table_extraction_flag == False:
                # Use the function to extract the text and format for each text element
                (line_text, format_per_line) = text_extraction(element)
                # Append the text of each line to the page text
                page_text.append(line_text)
                # Append the format for each line containing text
                line_format.append(format_per_line)
                page_content.append(line_text)
            else:
                # Omit the text that appeared in a table
                pass

        # Check the elements for images


    # Create the key of the dictionary
    dctkey = 'Page_'+str(pagenum)
    # Add the list of list as the value of the page key
    text_per_page[dctkey]= [page_text, line_format, page_content]

# Closing the pdf file object
  pdfFileObj.close()
  return text_per_page


dict_text = extract_pdf("appl_q2_2023.pdf")
print(dict_text)

# Display the content of the page


In [ ]:
comp_name + "-" + date_val

In [ ]:

def get_page_num(dict_text):
  page_num = ""
  index_trunc = 0
  for values in dict_text:
    if 'Questions And Answers\n' in dict_text[values][0]:
      page_num = values
      index_trunc = dict_text[values][0].index('Questions And Answers\n')
      break
  return [page_num, index_trunc]


def concat_text(dict_text):
  lst_speech = []
  trunc_set = get_page_num(dict_text)
  for values in dict_text:
    if values == trunc_set[0]:
      lst_speech = lst_speech + dict_text[values][0][2: trunc_set[1]]
      return lst_speech
    else:
      lst_speech = lst_speech + dict_text[values][0][2:-1]

def strng_check(comments_lst):
  if "{BIO" in comments_lst:
    return True
  else:
    return False

def get_all_index(dict_text):
  lst_text = concat_text(dict_text)
  page_indx = get_page_num(dict_text)
  bios_index = [i for i, n in enumerate(lst_text) if strng_check(n) == True]
  names_index =  [values - 1 for values in bios_index]
  names_values = [lst_text[names][0:-1] for names in names_index]
  bios_index.append(len(lst_text) - 1)

  author_note_dict = {}
  start = 0
  end = 0
  counter = 0
  for names in bios_index[0:-1]:
    start = bios_index[counter]
    end = bios_index[counter + 1]
    author_note_dict[names] = sent_tokenize(' '. join(lst_text[start + 1 : end]))
    counter = counter + 1

  comp_name = dict_text['Page_0'][0][0].split("\n")[1].split("(")[1][0:-1].split(" ")[0]
  date_val = dict_text['Page_0'][0][1][0:7]
  df_author_notes = pd.DataFrame({"Author_Name": names_values,
                                  "Words": author_note_dict.values(),
                                  "Report_Company": comp_name + "-" + date_val})

  df_author_notes = df_author_notes.explode('Words').reset_index(drop=True)


  return df_author_notes



#[i for i, n in enumerate((text_per_page["Page_0"][0] + text_per_page["Page_1"][0])) if strng_check(n) == True] # List comprehension




In [ ]:

df_all_notes = pd.DataFrame({'Author_Name': [],
                             'Words': [],
                             'Report_Company': []}, index = [])
for file in glob.glob("*.pdf"):
  print(file)
  pdf_extract = extract_pdf(file)
  df_current_report = get_all_index(pdf_extract)
  df_all_notes = pd.concat([df_all_notes, df_current_report])


In [ ]:
df_all_notes = df_all_notes.reset_index()
df_all_notes['Words'] = df_all_notes['Words'].apply(lambda x: ' '.join(x.split("\n")) if "\n" in x else x)
df_all_notes['Words']  = df_all_notes['Words'].apply(lambda x: '-'.join(x.split("- ")) if "- " in x else x)

In [ ]:
df_all_notes.head()

In [ ]:
df_all_notes.to_csv("/content/drive/My Drive/Colab Notebooks/report_text.csv")